# 3D tracking of points across video frames
This stage uses xy coordinates from synced left and right video frames of the same objects. It triangulates each point to produce xyz coordinates for the object (e.g. fish individual) for each video frame. Prior to this step, cameras should be calibrated in time and space. 

In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [2]:
### Load in left and right image points for triangulation (e.g. in csv)
left_points=os.path.abspath('Add_own_file_path') 
right_points=os.path.abspath('Add_own_file_path') 

left_points = pd.read_csv(left_points)
right_points= pd.read_csv(right_points)

### Change column names depending on own dframe - need a frame ID, an object/indv ID and xy coordinates for each
left_points = left_points[["frame_count", "name","x", "y" ]]
right_points = right_points[["frame_count", "name","x", "y" ]]

### Match left and right points that are visible in both left and right frames at same timepoint 
left_frame_id=left_points[["frame_count", "name"]]
right_frame_id=right_points[["frame_count", "name"]]
frame_id = pd.concat([left_frame_id, right_frame_id])
frame_id =frame_id.reset_index(drop=True)
frame_id_unique = frame_id[frame_id.duplicated(subset=["frame_count", "name"], keep=False)]

left = []
right = []
frame_id = []
for index, row in frame_id_unique.iterrows():
    ID = row['name']
    Frame = row['frame_count']
    frame_id.append(ID+"_Frame_"+str(Frame))
    l=left_points[((left_points['frame_count'] == Frame) & (left_points['name']== ID))].to_numpy()
    r=right_points[((right_points['frame_count'] == Frame) & (right_points['name']== ID))].to_numpy()
    left.append(l[:,[2,3]])
    right.append(r[:,[2,3]])
left=np.float32(left)
right=np.float32(right)

In [3]:
### Read in stereo callibration parameters
calibration_parameters=os.path.abspath('Add_own_file_path_to_extrinsic_properties')

### Load all camera parameters
cv_file=cv2.FileStorage()
cv_file.open(calibration_parameters, cv2.FileStorage_READ)
l_mtx=cv_file.getNode('l_mtx').mat()#cameramatrix
r_mtx=cv_file.getNode('r_mtx').mat()#cameramatrix
l_dist=cv_file.getNode('l_dist').mat()#distortion coeffecients
r_dist=cv_file.getNode('r_dist').mat()#distortion coeffecients
R=cv_file.getNode('R').mat()
T=cv_file.getNode('T').mat()
E=cv_file.getNode('E').mat()
F=cv_file.getNode('F').mat()

In [4]:
### Undistort points based on camera parameters 
def undistort_points(l_points, r_points, frame_id, l_mtx, r_mtx, l_dist, r_dist):
    
    l_points_undistort=[]
    r_points_undistort =[]
    
    for fname in frame_id:
        img_index=frame_id.index(fname)
        l_points_undistort.append(cv2.undistortPoints(l_points[img_index], l_mtx, l_dist, P=l_mtx)) 
        r_points_undistort.append(cv2.undistortPoints(r_points[img_index], r_mtx, r_dist, P=r_mtx))
        #specifying P = l_mtx/r_mtx puts points in real world points
    return(l_points_undistort, r_points_undistort, frame_id)   

In [ ]:
l_points_undistort, r_points_undistort, frame_id=undistort_points(left, right, frame_id, l_mtx, r_mtx, l_dist, r_dist)

In [5]:
### Triangulate points 

def triangulate_points(l_points_undistort, r_points_undistort, frame_id, l_mtx, r_mtx, R, T):
    
    projMat1 = l_mtx @ cv2.hconcat([np.eye(3), np.zeros((3,1))]) # Cam1 is the origin
    projMat2 = r_mtx @ cv2.hconcat([R, T]) # R, T from stereoCalibrate parameters
    frame_xyz=[]
    frame_index=[]
    
    for fname in frame_id:
        
        img_index=frame_id.index(fname)
        l_points=l_points_undistort[img_index]
        r_points=r_points_undistort[img_index]
        l_points=l_points[:,0]
        r_points=r_points[:,0]
        
        points3d=[]
        point=0
        while point < len(l_points):
            triangulation = cv2.triangulatePoints(projMat1, projMat2,l_points[point], r_points[point])
            triangulation = (triangulation[:3, :]/triangulation[3, :]).T
            points3d.append(triangulation)
            point=point+1
        frame_xyz.append(points3d)
        frame_index.append(fname)
    return(frame_xyz, frame_index) 

In [ ]:
frame_xyz, frame_index=triangulate_points(l_points_undistort, r_points_undistort, frame_id, l_mtx, r_mtx, R, T)
#This returns the XYZ coordinates of each individual point. These can be plotted and used in further analyses. 